<a href="https://colab.research.google.com/github/txvuong150764/Honour-Project/blob/master/Honours_Project_Multilabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/txvuong150764/Honour-Project-Dataset.git

Cloning into 'Honour-Project-Dataset'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 22 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (22/22), 2.45 MiB | 6.26 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
!pip install transformers datasets evaluate
!pip install -U accelerate
!pip install -U transformers
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

# BART

In [ ]:
import transformers
import torch
x = transformers.pipeline("summarization", model="facebook/bart-large-cnn", device_map='auto', truncation="only_first")

In [ ]:
def allenai_summarize(example):
  example['text'] = x(example['text'], max_length=130, min_length=30, do_sample=False)[0]['summary_text']
  return example

# Load dataset

In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd
import nltk
from nltk.corpus import stopwords  # Stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df_train = pd.read_excel("/content/Honour-Project-Dataset/Data_1194.xlsx")
df_train = pd.DataFrame(df_train)
df_train['label'] = df_train['Orientation'] - 1
df_train = df_train.drop(['ID', 'UT', 'TI', 'AB', 'Test', 'Orientation'], axis=1)
train_dataset = Dataset.from_pandas(df_train)

In [ ]:
df_test = pd.read_excel("/content/Honour-Project-Dataset/Data_100.xlsx")
df_test = pd.DataFrame(df_test)
df_test['text'] = df_test['title'] + " " + df_test['abstract']
df_test['label'] = df_test['Final annotation']
df_test = df_test.drop(['Number', 'title', 'abstract', 'Fan.Annotation', 'Gunnar.Annotation', 'Agreement 1st round', 'Lin.Annotation', 'Agreement 2nd round', 'Final annotation'], axis=1)
test_dataset = Dataset.from_pandas(df_test)

In [ ]:
train_dataset = train_dataset.rename_column('TI AB', 'text')

In [ ]:
train_valid_dataset = train_dataset.shuffle(seed=42).train_test_split(test_size=0.1)

In [ ]:
def convert(example):
  if example['label'] == 'Scientific':
    example['label'] = 0 if example['label'] == 'Scientific' else 1
  elif example['label'] == 'Societal':
    example['label'] = 1
  else:
    example['label'] = 2
  return example

In [ ]:
test_dataset = test_dataset.map(convert)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
train_dataset = DatasetDict({
    "train": train_valid_dataset['train'],
    "valid": train_valid_dataset['test'],
    "test": test_dataset
    })

In [ ]:
import re
import string

stpwrds = stopwords.words('english')
punc = string.punctuation
word_to_be_removed = ['Background', 'Methods', 'Conclusions', 'Limitations', 'Objectives', 'Results', 'Aims']

def remove(example):
  string = example['text']
  for word in word_to_be_removed:
    string = string.replace(word, '')
    string = string.replace(word.upper(), '')
    string = string.replace(word.lower(), '')
  pos = string.rfind("(C)")
  string = string[:pos]
  string = ' '.join([word for word in string.split() if word not in stpwrds])
  string = re.sub(r'[^\w\s]', '', string)
  string = re.sub("\(.*?\)", '', string)
  string = string.replace('  ', ' ')
  example['text'] = string
  return example

In [ ]:
train_dataset = train_dataset.map(remove)

Map:   0%|          | 0/1073 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
train_dataset['train'][13]

{'text': 'Implications of reserve size and forest connectivity for the conservation of wood-inhabiting fungi in Europe Among the factors that affect the conservation efficiency of protected areas, lack of connectivity is considered as one of the main problems. In this study, we assessed the influence of connectivity of European beech forest reserves on wood-inhabiting fungal communities, compared to the influence of local factors. To address this topic, we used a data matrix consisting of 344 fungal species on 1571 resource units (i.e. fallen beech logs, including their standing snags) sampled in 42 European beech forest reserves. Our results show that connectivity has significant effects.on wood-inhabiting fungal communities in European beech forest reserves, and that the effectiveness of reserves for maintaining the wood-inhabiting fungal diversity is compromised by habitat fragmentation. Connectivity at small scales (measured as the area of the reserve) had a strong influence on the

In [ ]:
train_dataset['train'][13]

{'text': 'Implications reserve size forest connectivity conservation woodinhabiting fungi Europe Among factors affect conservation efficiency protected areas lack connectivity considered one main problems In study assessed influence connectivity European beech forest reserves woodinhabiting fungal communities compared influence local factors To address topic used data matrix consisting 344 fungal species 1571 resource units ie fallen beech logs including standing snags sampled 42 European beech forest reserves Our show connectivity significant effectson woodinhabiting fungal communities European beech forest reserves effectiveness reserves maintaining woodinhabiting fungal diversity compromised habitat fragmentation Connectivity small scales measured area reserve strong influence occurrence indicator species also critical number species resource Connectivity larger scales connectivity surrounding beech forests seemed particularly critical community composition resource reserve levels I

# Proprocess

In [ ]:
id2label = {0: "Scientific", 1: "Societal", 2: "Combined"}
label2id = {"Scientific": 0, "Societal": 1, "Combined": 2}

In [ ]:
model_dict = {
    'scibert': 'allenai/scibert_scivocab_uncased',
    'ssci-bert': 'KM4STfulltext/SSCI-BERT-e2',
    'ssci-scibert': 'KM4STfulltext/SSCI-SciBERT-e2',
    'biobert': 'dmis-lab/biobert-v1.1',
}

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

def encoding(model_path):
  tokenizer = AutoTokenizer.from_pretrained(model_path)

  def preprocess_function(examples):
    text = examples["text"]
    return tokenizer(text, padding="max_length", truncation=True, max_length=512)

  datasets_tokenized = train_dataset.map(preprocess_function, batched=True)

  data_collator = DataCollatorWithPadding(tokenizer)

  return datasets_tokenized, data_collator, tokenizer

# Baseline

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

def training_baseline_process(model_name, model_path, datasets_tokenized, data_collator, tokenizer):
  model = AutoModelForSequenceClassification.from_pretrained(
    model_path, num_labels=3, id2label=id2label, label2id=label2id
  )

  for param in model.base_model.parameters():
    param.requires_grad = False

  training_args = TrainingArguments(
    output_dir=f"/content/Baseline/{model_name}",
    learning_rate=5e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=16,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps = 68,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=datasets_tokenized["train"],
      eval_dataset=datasets_tokenized["valid"],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
  )

  trainer.train()

In [ ]:
for model_name, model_path in model_dict.items():
  print(f"\033[2;32m Training baseline model: {model_name} --------------------------------------------------")
  datasets_tokenized, data_collator, tokenizer = encoding(model_path)
  training_baseline_process(model_name, model_path, datasets_tokenized, data_collator, tokenizer)

 Training baseline model: scibert --------------------------------------------------


Map:   0%|          | 0/1073 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

 Training baseline model: ssci-scibert --------------------------------------------------


Map:   0%|          | 0/1073 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

KeyboardInterrupt: ignored

# Training

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoModel, TrainingArguments, Trainer

def training_process(model_name, model_path, datasets_tokenized, data_collator, tokenizer):
  model = AutoModelForSequenceClassification.from_pretrained(
    model_path, num_labels=3, id2label=id2label, label2id=label2id
  )

  training_args = TrainingArguments(
    output_dir=f"/content/Output/{model_name}",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=16,
    weight_decay=1e-6,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps = 68,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=datasets_tokenized["train"],
      eval_dataset=datasets_tokenized["valid"],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
  )

  trainer.train()

In [ ]:
for model_name, model_path in model_dict.items():
  print(f"\033[2;32m Traing model {model_name} --------------------------------------------------")
  datasets_tokenized, data_collator, tokenizer = encoding(model_path)
  training_process(model_name, model_path, datasets_tokenized, data_collator, tokenizer)

# Custom Model

In [ ]:
from datasets import load_dataset,Dataset,DatasetDict
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

def preprocess_function(examples):
  text = examples["text"]
  return tokenizer(text, padding="max_length", truncation=True, max_length=512)

datasets_tokenized = train_dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/1073 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
datasets_tokenized.set_format("torch",columns=["input_ids", "attention_mask", "label"])

In [ ]:
datasets_tokenized

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1073
    })
    valid: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 120
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [ ]:
class CustomModel(nn.Module):
  def __init__(self, model, num_labels):
    super(CustomModel,self).__init__()
    self.num_labels = num_labels

    #Load Model with given checkpoint and extract its body
    self.model = AutoModel.from_pretrained(model,config=AutoConfig.from_pretrained(model, output_attentions=True,output_hidden_states=True))
    self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(768,num_labels) # load and initialize weights

  def forward(self, input_ids=None, attention_mask=None,labels=None):
    #Extract outputs from the body
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

    #Add custom layers
    sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state

    logits = self.classifier(sequence_output[:,0,:].view(-1,768)) # calculate losses

    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

    return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states,attentions=outputs.attentions)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=CustomModel(model='allenai/scibert_scivocab_uncased',num_labels=3).to(device)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    datasets_tokenized["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    datasets_tokenized["valid"], batch_size=8, collate_fn=data_collator
)

In [ ]:
from transformers import AdamW,get_scheduler

optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 16
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

2160


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from datasets import load_metric
metric = load_metric("accuracy")

<ipython-input-29-a8d3d6c893c5>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epochs * len(eval_dataloader)))


for epoch in range(num_epochs):
  model.train()
  for batch in train_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      progress_bar_train.update(1)

  model.eval()
  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
    progress_bar_eval.update(1)

  print(metric.compute())

  0%|          | 0/2160 [00:00<?, ?it/s]

  0%|          | 0/240 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'accuracy': 0.5916666666666667}
{'accuracy': 0.6333333333333333}
{'accuracy': 0.6416666666666667}
{'accuracy': 0.6833333333333333}
{'accuracy': 0.5916666666666667}
{'accuracy': 0.65}
{'accuracy': 0.6666666666666666}
{'accuracy': 0.6916666666666667}
{'accuracy': 0.65}
{'accuracy': 0.6583333333333333}
{'accuracy': 0.65}
{'accuracy': 0.6833333333333333}
{'accuracy': 0.6333333333333333}
{'accuracy': 0.6833333333333333}
{'accuracy': 0.675}
{'accuracy': 0.675}


In [ ]:
model.eval()

test_dataloader = DataLoader(
    datasets_tokenized["test"], batch_size=4, collate_fn=data_collator
)

pre = []

for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    pre.append(predictions)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.53}

In [ ]:
torch.save(model.state_dict(), '/gdrive/MyDrive/Honour Project/bio-bert.pth')

In [ ]:
datasets_tokenized["test"]['label']

tensor([1, 1, 2, 2, 2, 1, 1, 0, 1, 2, 1, 1, 0, 0, 1, 1, 1, 2, 1, 0, 1, 2, 2, 2,
        1, 2, 1, 2, 2, 1, 2, 2, 0, 1, 0, 2, 1, 1, 2, 0, 2, 1, 1, 0, 2, 0, 0, 0,
        1, 2, 2, 2, 2, 1, 1, 0, 1, 0, 2, 2, 2, 1, 2, 1, 2, 1, 1, 1, 0, 0, 1, 0,
        0, 2, 1, 1, 1, 2, 1, 2, 0, 0, 2, 1, 2, 0, 1, 1, 1, 1, 0, 1, 2, 1, 2, 0,
        1, 0, 0, 0])

In [ ]:
pre

[tensor([1, 1, 2, 0], device='cuda:0'),
 tensor([1, 2, 1, 2], device='cuda:0'),
 tensor([1, 1, 1, 1], device='cuda:0'),
 tensor([0, 0, 0, 0], device='cuda:0'),
 tensor([0, 0, 1, 0], device='cuda:0'),
 tensor([1, 0, 2, 0], device='cuda:0'),
 tensor([1, 0, 1, 0], device='cuda:0'),
 tensor([1, 0, 1, 0], device='cuda:0'),
 tensor([0, 2, 0, 0], device='cuda:0'),
 tensor([0, 1, 0, 0], device='cuda:0'),
 tensor([0, 1, 1, 0], device='cuda:0'),
 tensor([2, 0, 0, 0], device='cuda:0'),
 tensor([1, 0, 0, 0], device='cuda:0'),
 tensor([0, 1, 1, 0], device='cuda:0'),
 tensor([2, 0, 0, 1], device='cuda:0'),
 tensor([0, 1, 0, 0], device='cuda:0'),
 tensor([0, 0, 1, 1], device='cuda:0'),
 tensor([0, 0, 0, 0], device='cuda:0'),
 tensor([0, 0, 1, 0], device='cuda:0'),
 tensor([1, 0, 2, 2], device='cuda:0'),
 tensor([0, 0, 2, 2], device='cuda:0'),
 tensor([2, 0, 2, 1], device='cuda:0'),
 tensor([1, 0, 0, 1], device='cuda:0'),
 tensor([0, 1, 2, 0], device='cuda:0'),
 tensor([1, 0, 0, 0], device='cuda:0')]

# Inference


In [ ]:
from transformers import pipeline

for model_name, model_path in model_dict.items():
  classifier = pipeline("text-classification", model=f"/content/Baseline/{model_name}/checkpoint-1088", max_length=512, truncation=True)

  cnt=0
  for test in train_dataset['test']:
    pred = classifier(test['text'])[0]['label']
    if label2id[pred] == test['label']:
      cnt += 1

  print(f"Accuracy for {model_name} is: {cnt/len(train_dataset['test'])}")